### 타깃
[naver news:공공자전거](https://search.naver.com/search.naver?&where=news&query=%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&start=1&refresh_start=0)

#### Ref
- [selenium attribute](https://stackoverflow.com/questions/30324760/how-to-get-attribute-of-element-from-selenium?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)
- [~~selenium multiple class~~](https://stackoverflow.com/questions/21713280/find-div-element-by-multiple-class-names?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa) selenium옛버전. 이젠 contains 나 findElement는 안됨

In [63]:
# coding: utf-8
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os.path
from os import rename
from os import listdir
from os import mkdir

import time

import csv

In [2]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import re

In [3]:
def startWeb(numPage=1):
    """
    네이버 뉴스
    검색어: 공공자전거
    옵션유지: 켜짐. mynews=1
    @return driver
    """
    driver= webdriver.Chrome('lib/chromedriver.exe')
    paging= str(1+10*(numPage-1))
    # caution: "\ is before &"
    url= "https://search.naver.com/search.naver?&where=news\
    &query=%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0&sm=tab_pge\
    &sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=\
    &nso=so:r,p:all,a:all&mynews=1&start="+paging+"&refresh_start=0"
    driver.get(url)
    
    try:
        element= WebDriverWait(driver, 10).until(
            # is comming content?
            EC.presence_of_element_located((By.XPATH, '//li'))
        )
    except Exception as e:
        print('error',e)
    return driver

#### 언론사 선택

In [4]:
def searchOptPress():
    """
    일간지, 뉴시스, 연합뉴스, 한국경제TV, JTBC, 시사IN
    """
    #검색옵션
    try:
        element= WebDriverWait(driver, 10).until(
            # is comming content?
            EC.presence_of_element_located((By.XPATH, '//a[@id="_search_option_btn"]'))
        )
    except Exception as e:
        print('error',e)
    element.click()

    #언론사
    xpath1= '//li[@id="news_popup"]'
    element= driver.find_element_by_xpath(xpath1)
    element.click()

    # #전체선택 해제. '18.4.24 부로 전체선택옵션이 없어졌다 ㄷㄷ
    # xpath2= '//input[@id="select_all_order"]'
    # element= driver.find_element_by_xpath(xpath2)
    # element.click()

    #일간지 선택
    xpath3= '//input[@id="ca_p1"]'
    element= driver.find_element_by_xpath(xpath3)
    element.click()
    
    somePress= ["뉴시스", "연합뉴스", "한국경제TV", "JTBC", "시사IN"]
    for press in somePress:
        xpath= '//label[@title="'+press+'"]'
        element= driver.find_element_by_xpath(xpath)
        element.click()
        
    # 확인
    xpathChk= '//button[@class="impact _submit_btn"]'
    element= driver.find_element_by_xpath(xpathChk)
    element.click()
    
    try:
        element= WebDriverWait(driver, 10).until(
            # is comming content?
            EC.presence_of_element_located((By.XPATH, '//li'))
        )
    except Exception as e:
        print('error',e)

#### 링크 얻기

In [5]:
def movePage(numPage):
    paging= str(1+10*(numPage-1))
    url= "https://search.naver.com/search.naver?&where=news\
    &query=%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0&sm=tab_pge\
    &sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=\
    &nso=so:r,p:all,a:all&mynews=1&start="+paging+"&refresh_start=0"
    driver.get(url)
    
    try:
        element= WebDriverWait(driver, 10).until(
            # is comming content?
            EC.presence_of_element_located((By.XPATH, '//div[@class="thumb"]'))
        )
    except Exception as e:
        print('error',e)

In [6]:
def saveLink(listLink):
    #caution: not '_sp_each_title' but ' _sp_each_title'
    #amazing: there are no duplicated. why ??
    
    xpathLink1= '//a[@class=" _sp_each_title"]'
    elements= driver.find_elements_by_xpath(xpathLink1)
    for element in elements:
        listLink.append(element.get_attribute("href"))
        
    xpathLink2= '//a[@class="_sp_each_url _sp_each_title"]'
    elements= driver.find_elements_by_xpath(xpathLink2)
    for element in elements:
        listLink.append(element.get_attribute("href"))
        
    return listLink

#### 각 링크의 내용

In [7]:
def movePageLink(link):
    driver.get(link)
    try:
        element= WebDriverWait(driver, 10).until(
            # is coming a content?
            EC.presence_of_element_located((By.XPATH, '//div'))
        )
    except Exception as e:
        print('error',e)

#### 전제(낙관)
같은 종류의 사이트라면 같은 xpath를 쓸 것이다

In [8]:
def mkDictNewsXpath():
    """
    link scraping 결과에 unique() 하고, 의뭉스런 사이트를 직접 들어가서
    해당 언론사가 필터링한 언론사 맞는지 확인한 결과
    
    xpath가 틀렸다면 여기서 정정!
    """
    link= ['news.khan.co', 'www.khan.co', 'h2.khan.co', 'news.kmib.co', 'www.naeil.com', 'news.donga.com', 'www.donga.com', 'realestate.donga.com', 'bizn.donga.com', 'travel.donga.com', 'studio.donga.com', 'yachtpia.donga.com', 'it.donga.com', 'www.m-i.kr', 'www.munhwa.com', 'www.seoul.co', 'ntn.seoul.co', 'stv.seoul.co', 'www.segye.com', 'local.segye.com', 'www.segyefn.com', 'economysegye.segye.com', 'fn.segye.com', 'www.asiatoday.co', 'news.chosun.com', 'biz.chosun.com', 'car.chosun.com', 'life.chosun.com', 'businessnews.chosun.com', 'news.joins.com', 'realestate.joins.com', 'www.hani.co', 'babytree.hani.co', '2korea.hani.co', 'www.hankookilbo.com', 'www.newsis.com', 'sports.news.naver', 'news.naver.com', 'app.yonhapnews.co', 'www.wowtv.co', 'www.wownet.co', 'news.wowtv.co', 'sports.wowtv.co', 'wowstar.wowtv.co', 'news.jtbc.co', 'www.sisain.co', 'www.sisainlive.com']
    dupCnt= [3,1,1,8,1,1,3,5,1,5,2,3,1,2,2,5,1,2]
    xpaths= ['//div[@id="articleBody"]', '//div[@id="articleBody"]', '//div[@class="articleArea"]', '//div[@id="ct"]', '//div[@class="content"]', '//div[@id="NewsAdContent"]', '//div[@id="article_content"]', '//div[@id="article_txt"]', '//div[@id="font"]', '//div[@id="news_body_id"]/div[@class="par"]', '//div[@id="article_body"]', '//div[@class="article-text-font-size"]', '//article[@id="article-body"]','//div[@itemprop="articleBody"]', '//div[@id="articleWrap"]', '//div[@id="divNewsContent"]', '//div[@class="article_content"]', '//div[@itemprop="articleBody"]']
    xpaths= np.array(xpaths)

    xpaths_s= list(xpaths.repeat(dupCnt))

    dictLink= dict(zip(link,xpaths_s))
    return dictLink

In [9]:
def mkDictNewsPress():
    """
    해당 링크가 어느 신문사인지
    """
    link= ['news.khan.co', 'www.khan.co', 'h2.khan.co', 'news.kmib.co', 'www.naeil.com', 'news.donga.com', 'www.donga.com', 'realestate.donga.com', 'bizn.donga.com', 'travel.donga.com', 'studio.donga.com', 'yachtpia.donga.com', 'it.donga.com', 'www.m-i.kr', 'www.munhwa.com', 'www.seoul.co', 'ntn.seoul.co', 'stv.seoul.co', 'www.segye.com', 'local.segye.com', 'www.segyefn.com', 'economysegye.segye.com', 'fn.segye.com', 'www.asiatoday.co', 'news.chosun.com', 'biz.chosun.com', 'car.chosun.com', 'life.chosun.com', 'businessnews.chosun.com', 'news.joins.com', 'realestate.joins.com', 'www.hani.co', 'babytree.hani.co', '2korea.hani.co', 'www.hankookilbo.com', 'www.newsis.com', 'sports.news.naver', 'news.naver.com', 'app.yonhapnews.co', 'www.wowtv.co', 'www.wownet.co', 'news.wowtv.co', 'sports.wowtv.co', 'wowstar.wowtv.co', 'news.jtbc.co', 'www.sisain.co', 'www.sisainlive.com']
    dupCnt= [3,1,1,8,1,1,3,5,1,5,2,3,1,2,2,5,1,2]
    press= ['경향신문', '국민일보', '내일신문', '동아일보', '매일일보', '문화일보', '서울신문', '세계일보', '아시아투데이', '조선일보', '중앙일보', '한겨레', '한국일보', '뉴시스', '연합뉴스', '한국경제TV ', 'JTBC', '시사IN']
    press= np.array(press)

    press_s= list(press.repeat(dupCnt))

    dictPress= dict(zip(link,press_s))
    return dictPress

In [10]:
dictLink= mkDictNewsXpath()
def whatLinksXpath(link):
    link3piece= '.'.join(link.split('/')[2].split('.')[0:3])
    xpath= dictLink[link3piece]
    return xpath

In [11]:
dictPress= mkDictNewsPress()
def whatLinksPress(link3piece):
    #link3piece= '.'.join(link.split('/')[2].split('.')[0:3])
    press= dictPress[link3piece]
    return press

In [12]:
def rmSpecialChr(weirdStr):
    """
    @param string
    @return string. #특문제거, \n -> ___
    """
    regPattLine= '\n'
    res= re.sub(regPattLine,'___',weirdStr)
    regPatt= '[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9()": ]|[[\u3131-\u318E\uAC00-\uD7A3]]'
    res= re.sub(regPatt,'',res)
    res= re.sub('\\s+', ' ', res) # 공백 제거 해,말아? 하자. 안 하면 헬일듯
    return res

In [13]:
def getContent(link):
    """
    @return conBodyText
    """
    xpathBody= whatLinksXpath(link)
    #print(xpathBody)
    
    try:
        conBody= WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, xpathBody))
        )
        conBodyText= rmSpecialChr(conBody.text)
    except Exception as e:
        print('no match xpath',e)
        conBodyText= ""
        pass
    
    return conBodyText

In [66]:
if not os.path.exists('data'):
    mkdir("data")

###### 세션 종료, 자원반납

In [14]:
# driver.close()
# driver.quit()

## test_성공
1페이지에서 link 목록 따오기

In [15]:
# TEST: link list from page-1
driver= startWeb()

In [16]:
searchOptPress()

In [17]:
numPage= 1
movePage(numPage)

In [18]:
listLink= list([])
listLink= saveLink(listLink)

listLink

['http://www.newsis.com/view/?id=NISX20180421_0000288016&cID=10803&pID=10800',
 'http://www.seoul.co.kr/news/newsView.php?id=20180417014003&wlog_tag3=naver',
 'http://www.hankookilbo.com/v/0e2f312b60b84e099b87d04436fc3308',
 'http://bizn.donga.com/realestate/3/all/20180416/89637182/2',
 'http://news.joins.com/article/olink/22127647',
 'http://news.joins.com/article/olink/22120203',
 'http://news.chosun.com/site/data/html_dir/2018/04/10/2018041000149.html',
 'http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.aspx?c=AKR20180408025600004&did=1195m',
 'http://www.asiatoday.co.kr/view.php?key=20180422010012584',
 'http://www.asiatoday.co.kr/view.php?key=20180418010010605']

In [19]:
driver.close()
driver.quit()

## test _성공
link 하나에서 뉴스 내용 퍼오기

In [20]:
# TEST: news content from link-1
driver= startWeb()

In [21]:
searchOptPress()

In [22]:
# numPage= 2
# movePage(numPage)

In [23]:
dictLink= mkDictNewsXpath() # 최초 1번만 만들면 된다

linkEx= 'http://www.newsis.com/view/?id=NISX20180421_0000288016&cID=10803&pID=10800'

In [24]:
movePageLink(linkEx)

In [25]:
contents= getContent(linkEx)
contents

'수원뉴시스김경호 기자 수원시 무인 대여자전거수원뉴시스김경호 기자 경기 수원시가 지난해 전국 최초로 도입한 스테이션 없는 무인대여자전거가 올해 상반기 중 대폭 늘어난다 수원시는 무인대여자전거 950대를 운영하는 공유자전거 업체 모바이크(Mobike)가 상반기 안에 4050대를 추가한다고 21일 밝혔다 5월 중 1400대 6월 중 2650대를 배치할 계획이다 새롭게 배치되는 자전거는 뉴라이트 모델로 기존 자전거보다 성능이 개선됐다 자전거 무게를 6(2216) 줄였고 잠금장치 배터리 방전을 방지하기 위한 태양광 패널을 부착했다 여성아동도 쉽게 탈 수 있도록 안장 높이 조절 범위를 늘렸다 회원 가입할 때 내야 했던 보증금(5000원)은 23일부터 없애 이용자 부담을 줄인다 또 국내 모든 신용카드로 이용료를 결제할 수 있도록 결제 시스템을 개선할 계획이다 수원시는 5월부터 무인대여 자전거 주차구역 관리(자전거 정리정돈거치대 청소방치자전거 처리 등)를 민간기관에 위탁해 시민 불편을 최소화할 계획이다 자전거 이용경로를 알려주는 빅데이터를 활용해 요소요소에 자전거 주차장을 확보하고 자전거도로 정비 우선순위를 정할 예정이다 무인대여자전거 시스템은 자전거 거치대 무인 정보안내시스템 등이 필요 없다 스마트폰을 활용해 GPS가 장착된 자전거를 수원시 곳곳에 있는 자전거 주차공간에서 간편하게 대여하고 반납할 수 있다 현재 수원시에서는 공유자전거 업체인 모바이크와 오바이크(oBike)가 무인대여 자전거를 운영하고 있다 무인대여 자전거를 이용하려면 먼저 스마트폰 플레이스토어(안드로이드폰) 앱스토어(아이폰)에서 오바이크나 모바이크를 검색해 애플리케이션을 내려받아야 한다 무인자전거 앱을 활성화하면 주변에 있는 자전거 위치를 알려주는 지도가 나온다 자전거가 있는 장소를 찾아가 앱 하단 잠금 해제를 누른 후 스마트폰으로 자전거 핸들 사이에 부착된 큐알(QR)코드를 스캔하면 잠금이 해제된다 자전거를 타고 목적지까지 이동한 뒤 목적지 주변에 있는 공공자전거 주차공간(노면에 표시)이나 자전거 거치

---

## 실행- 링크얻기_완료
링크가 필요하면 또 돌리되, 시간이 제법 걸린다

In [26]:
## Run
# driver= startWeb()
# searchOptPress()

In [27]:
## data/naverNewsSomePressLinks.csv 에 저장 완료!
# numPages= range(1,400)
# listLink= list([])

# for numPage in numPages:
#     print(numPage)
#     movePage(numPage)
#     listLink= saveLink(listLink)

#print(len(listLink))
# listLink

In [28]:
# driver.close()
# driver.quit()

# 저장&읽기

### 저장 from Scraping
- [dictionary to csv](https://stackoverflow.com/questions/8685809/python-writing-a-dictionary-to-a-csv-file-with-one-line-for-every-key-value?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)
    - [writerow one line feed](https://stackoverflow.com/questions/3191528/csv-in-python-adding-an-extra-carriage-return?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)

In [29]:
# # scraping 직후. 옵션외 링크가 좀 섞여있다
# DataFrame(listLink).to_csv("data/naverNewsSomePressLinks.csv", index= False, header=None)

#### 읽기
scraping 직후. 옵션외 링크가 좀 섞여있다

In [30]:
# linkRead= pd.read_csv("data/naverNewsSomePressLinks.csv", header=None)

# linkRead= list(linkRead[0])

# print(len(linkRead))

# linkRead[30:35]

#### 옵션 외 언론사 지우기
133개 지움

In [31]:
# df= DataFrame(linkRead)

# exceptList= ['www.kukinews.com', 'news.kukinews.com', 'www.seouland.com', 'nownews.seoul.co', 'nownewstv.seoul.co', 'www.sporbiz.co', 'www.beautyhankook.com']
# for exLink in exceptList:
#     df= df[df[0].map(lambda x: (exLink not in x))]
    
# print(len(df))

# 다시 저장 though exceptList
이게 제대로된 링크 목록

In [32]:
df.to_csv("data/naverNewsDailyPlusPressLinks.csv", index= False, header=None)

### 읽기
광고? 제거된 링크

In [33]:
linkNewsDailyPlus= pd.read_csv('data/naverNewsDailyPlusPressLinks.csv',
                               header=None)

print(linkNewsDailyPlus.shape)
print(type(linkNewsDailyPlus[0]))
linkNewsDailyPlus.head(3)

(3857, 1)
<class 'pandas.core.series.Series'>


0
0  http://www.newsis.com/view/?id=NISX20180421_00...
1  http://www.seoul.co.kr/news/newsView.php?id=20...
2  http://www.hankookilbo.com/v/0e2f312b60b84e099...

In [34]:
linkNewsDailyPlus_list= list(linkNewsDailyPlus[0])
len(linkNewsDailyPlus_list)

3857

# 실행- 내용얻기

In [35]:
driver.close()
driver.quit()

#### 링크에서 각 값 따오기-> DataFrame 만들기

In [36]:
linkNewsDailyPlus= pd.read_csv('data/naverNewsDailyPlusPressLinks.csv',
                               header=None)
linkNewsDailyPlus_list= list(linkNewsDailyPlus[0])
print(len(linkNewsDailyPlus_list))

bodyList= list([])
pressList3pie= list([])

dictLink= mkDictNewsXpath() # 최초 1번만 만들면 된다. driver와 무관

3857


In [37]:
driver= startWeb()
searchOptPress()

for i, link in enumerate(linkNewsDailyPlus_list):
    #if (0== i)| (1==i):
    #print(link)
    print(i)

    movePageLink(link)
    body= getContent(link)
    bodyList.append(body)

    link3piece= '.'.join(link.split('/')[2].split('.')[0:3])
    pressList3pie.append(link3piece)

In [38]:
newsData= { 
    'link3pie': pressList3pie,
    'zbody':bodyList
}
dfNews= pd.DataFrame(data= newsData)

### 저장

In [40]:
dfNews.to_csv('data/naverNewsContent.csv', index= False, encoding= 'cp949')

NameError: name 'dfNews' is not defined

### 뭔 신문사인지?

In [41]:
dictPress= mkDictNewsPress()

In [42]:
dfNews['link3pie'].map(whatLinksPress).value_counts()

In [43]:
list(dfNews['link3pie'].map(whatLinksPress).unique())

In [44]:
dfNews['link3pie'].value_counts()

## 읽기

In [2]:
import pandas as pd

In [45]:
dfNC= pd.read_csv("data/naverNewsContent.csv", encoding= 'cp949')

In [46]:
dfNC.columns

#### 문자열 포함된 거 찾기

In [47]:
def findStr(inputStr):
    tradein= lambda x:inputStr in x
    return df[df.zbody.map(tradein)]

In [48]:
inputStr= '음식'
findStr(inputStr).to_csv('data/newsfind'+inputStr+'.csv', index= False, encoding= 'cp949')

---

---

---

#### 어느 사이트인지 판단
18개 xpath 중에 무엇일지

In [27]:
# linkRead= pd.read_csv("data/naverNewsDailyPlusPressLinks.csv", header=None)
# linkRead= list(linkRead[0])

# pressList3piece= ['.'.join(x.split('/')[2].split('.')[0:3]) for x in linkRead]

# print(Series(pressList3piece).unique())
# print(len(Series(pressList3piece).unique()))
# print(Series(pressList3piece).value_counts())

# pressList2piece= ['.'.join(x.split('/')[2].split('.')[1:3]) for x in linkRead]

# print(Series(pressList2piece).unique())
# print(len(Series(pressList2piece).unique()))
# print(Series(pressList2piece).value_counts())

# # 18
# # 개 언론사만 골라서했는데 27개 나옴. > 무슨무슨링크랑 회사랑 매치되는지 알아야
# # - news.naver.com : 연합뉴스
# # - sports.news.naver : 뉴시스

# dfLinkRead= DataFrame(linkRead)
# dfLinkEx= dfLinkRead[dfLinkRead[0].map(lambda x:'naver.com' in x)]
# print(dfLinkEx)

# # sports.news.naver  3
# # news.naver.com 928
# print(len(dfLinkEx))

In [34]:
# # naver 있는 게 왜이리 많은지 보기
# dfLinkEx.to_csv("data/naver_in_link.csv", index= False, header=None)

#### link~xpath Dictionary 만드는 코드 백업

In [41]:
# df= pd.read_csv('data/naverNewsSomePressLinks.csv', index_col=False, header=None)
#len(df)

# ## 긁으려는 거 외 링크 지우기
# # 133개 지움
# exceptList= ['www.kukinews.com', 'news.kukinews.com', 'www.seouland.com', 'nownews.seoul.co', 'nownewstv.seoul.co', 'www.sporbiz.co', 'www.beautyhankook.com']
# for exLink in exceptList:
#     df= df[df[0].map(lambda x: (exLink not in x))]

#df
#len(df)
#df[0].map(lambda x: (exceptList[0] in x)).sum()

## dict 만들기
# key: http빼고 사이트 앞 3어절, value: XPATH
# link= ['news.khan.co', 'www.khan.co', 'h2.khan.co', 'news.kmib.co', 'www.naeil.com', 'news.donga.com', 'www.donga.com', 'realestate.donga.com', 'bizn.donga.com', 'travel.donga.com', 'studio.donga.com', 'yachtpia.donga.com', 'it.donga.com', 'www.m-i.kr', 'www.munhwa.com', 'www.seoul.co', 'ntn.seoul.co', 'stv.seoul.co', 'www.segye.com', 'local.segye.com', 'www.segyefn.com', 'economysegye.segye.com', 'fn.segye.com', 'www.asiatoday.co', 'news.chosun.com', 'biz.chosun.com', 'car.chosun.com', 'life.chosun.com', 'businessnews.chosun.com', 'news.joins.com', 'realestate.joins.com', 'www.hani.co', 'babytree.hani.co', '2korea.hani.co', 'www.hankookilbo.com', 'www.newsis.com', 'sports.news.naver', 'news.naver.com', 'app.yonhapnews.co', 'www.wowtv.co', 'www.wownet.co', 'news.wowtv.co', 'sports.wowtv.co', 'wowstar.wowtv.co', 'news.jtbc.co', 'www.sisain.co', 'www.sisainlive.com']
# dupCnt= [3,1,1,8,1,1,3,5,1,5,2,3,1,2,2,5,1,2]
# xpaths= ['//div[@id="articleBody"]', '//div[@id="articleBody"]', '//div[@class="articleArea"]', '//div[@id="ct"]', '//div[@class="content"]', '//div[@id="NewsAdContent"]', '//div[@id="article_content"]', '//div[@id="article_txt"]', '//div[@id="font"]', '//div[@id="news_body_id"]/div[@class="par"]', '//div[@id="article_body"]', '//div[@class="article-text-font-size"]', '//article[@id="article-body"]', '//div[@id="#textBody"]', '//div[@id="articleWrap"]', '//div[@id="divNewsContent"]', '//div[@class="article_content"]', '//div[@itemprop="articleBody"]']
# xpaths= np.array(xpaths)
# print('xpaths렝',len(xpaths))
# print('dupCnt렝',len(dupCnt))

# xpaths_s= list(xpaths.repeat(dupCnt))
# xpaths_s
# print('link>> ',link)
# print('xpaths_s>> ',xpaths_s)

# dictLink= dict(zip(link,xpaths_s))
#dictLink

# ## xpath 얻기
# sriLink= df[0].map(lambda x: '.'.join(x.split('/')[2].split('.')[0:3]))
# #sriLink
# #sriLink.value_counts()

# ## 미리 만들어둔 list ( link ~ xpath ) ㄴㄴ
# xpathList= list([])
# for link in sriLink:
#     xpathList.append(dictLink[link])
# xpathList